In [1]:
import pyspark

from pyspark.sql import SparkSession
from pyspark.sql import functions as f


## 1. From the given Car details dataset, compute the average weight of American cars for each make

In [2]:
import findspark
findspark.init()

from pyspark import SparkContext
sc=SparkContext("local[*]","Cars-Case study")

path='C:/Users/91784/Downloads/pyspark-master (1)/pyspark-master/data_git/cars.tsv'

def seqfn(z,v):
    return (z[0]+v,z[1]+1)

def combfn(a,b):
    return(a[0]+b[0],a[1]+b[1])

rdd_out=sc.textFile(path, 4) \
            .map(lambda s : s.split("\t")) \
            .filter(lambda x:x[-1]=="American") \
            .map(lambda l: (l[0],l[6])) \
            .mapValues(int) \
            .aggregateByKey((0,0),seqfn, combfn) \
            .mapValues(lambda x: x[0]/x[1]) \
            .coalesce(1)

rdd_out.collect()


sc.stop()

-----

## 2. Compute the total SUM,Avg, Max, Count of the salevalue for each customer for each month

In [13]:
spark1 = SparkSession.builder.appName('Retail').getOrCreate()
df1=spark1.read.csv('data/Online Retail.csv', header=True, inferSchema=True)
df1.show(2)
df1.printSchema()

+---------+---------+--------------------+--------+--------------+---------+----------+--------------+
|InvoiceNo|StockCode|         Description|Quantity|   InvoiceDate|UnitPrice|CustomerID|       Country|
+---------+---------+--------------------+--------+--------------+---------+----------+--------------+
|   536365|   85123A|WHITE HANGING HEA...|       6|12/1/2010 8:26|     2.55|     17850|United Kingdom|
|   536365|    71053| WHITE METAL LANTERN|       6|12/1/2010 8:26|     3.39|     17850|United Kingdom|
+---------+---------+--------------------+--------+--------------+---------+----------+--------------+
only showing top 2 rows

root
 |-- InvoiceNo: string (nullable = true)
 |-- StockCode: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Quantity: integer (nullable = true)
 |-- InvoiceDate: string (nullable = true)
 |-- UnitPrice: double (nullable = true)
 |-- CustomerID: integer (nullable = true)
 |-- Country: string (nullable = true)



In [14]:
# a. Filter all the customers with null value

df1.createOrReplaceTempView("DATA")

df2=spark1.sql("SELECT * FROM DATA where CustomerID IS NOT NULL")
# df_retail.show()

In [20]:
#b. SaleValue is derived as UnitPrice*Quantity
df2=df2.withColumn('SaleValue', round(df2.Quantity * df2.UnitPrice,2))
# df_retail.show()

In [21]:
#c.Create the month as an additional derived column from InvoiceDAte in YYYY-MM format(ex-2022-01)

df2=df2.select('InvoiceNo','StockCode','Description','Quantity','UnitPrice','CustomerID','Country','SaleValue',
    to_timestamp(f.col('InvoiceDate'), 'M/d/y H:m').alias('InvoiceDate')
)

df2=df2.withColumn("InvoiceMonth", f.from_unixtime(f.unix_timestamp(df2.InvoiceDate), "yyyy-MM"))
df2.show(3)
df2.printSchema()

+---------+---------+--------------------+--------+---------+----------+--------------+---------+-------------------+------------+
|InvoiceNo|StockCode|         Description|Quantity|UnitPrice|CustomerID|       Country|SaleValue|        InvoiceDate|InvoiceMonth|
+---------+---------+--------------------+--------+---------+----------+--------------+---------+-------------------+------------+
|   536365|   85123A|WHITE HANGING HEA...|       6|     2.55|     17850|United Kingdom|     15.3|2010-12-01 08:26:00|     2010-12|
|   536365|    71053| WHITE METAL LANTERN|       6|     3.39|     17850|United Kingdom|    20.34|2010-12-01 08:26:00|     2010-12|
|   536365|   84406B|CREAM CUPID HEART...|       8|     2.75|     17850|United Kingdom|     22.0|2010-12-01 08:26:00|     2010-12|
+---------+---------+--------------------+--------+---------+----------+--------------+---------+-------------------+------------+
only showing top 3 rows

root
 |-- InvoiceNo: string (nullable = true)
 |-- StockCo

In [24]:
#d. Shown below is a sample of the output
df2.groupBy("CustomerID","InvoiceMonth").agg(round(sum("SaleValue"),2).alias("sum"), \
         round(avg("SaleValue"),2).alias("avg"), \
         max("SaleValue").alias("max"), \
         count("SaleValue").alias("count") \
     ) \
    .show()

+----------+------------+-------+------+------+-----+
|CustomerID|InvoiceMonth|    sum|   avg|   max|count|
+----------+------------+-------+------+------+-----+
|     15291|     2010-12|  648.9|108.15| 166.8|    6|
|     12763|     2010-12| 320.08| 20.01|  60.0|   16|
|     14825|     2010-12| 396.43| 22.02|  85.0|   18|
|     14355|     2010-12|  174.4| 15.85|  25.5|   11|
|     17404|     2010-12| 2646.3| 98.01|1188.0|   27|
|     17343|     2010-12| 193.64| 11.39|  35.7|   17|
|     14978|     2011-01| 689.54|  9.32|  25.5|   74|
|     14176|     2011-01|  121.2|  30.3|  35.4|    4|
|     17511|     2010-12|6634.76| 65.05| 232.0|  102|
|     17567|     2010-12| 517.95| 19.18|  39.6|   27|
|     13030|     2010-12| 427.74|  7.13| 44.85|   60|
|     13802|     2010-12|-117.45|-58.73| -7.95|    2|
|     12682|     2011-01| 835.11| 20.88|  72.0|   40|
|     12647|     2010-12| 1140.7| 21.52|  90.0|   53|
|     15021|     2010-12| 189.64|  3.33|  23.6|   57|
|     14176|     2010-12| 15

In [ ]:
# df2.groupBy("CustomerID","InvoiceMonth").agg(round(sum("SaleValue"),2).alias("sum"), \
#          round(avg("SaleValue"),2).alias("avg"), \
#          max("SaleValue").alias("max"), \
#          count("SaleValue").alias("count") \
#      ) \
#     .show()



# spark1.sql("select sum(SaleValue) as Invoicesum, avg(SaleValue) as Invoinceavg, max(SaleValue) as InvoiceMax, count(SaleValue) as InvoiceCount from DATA " +
#           "group by CustomerID,InvoiceMonth ").show()

# spark1.sql("select sum(SaleValue) as Invoicesum from DATA").show()

----

## 3. Find out the top 10 customers with highest salevalue in the year 2011. 

In [85]:
df3=spark1.read.csv('data/Online Retail.csv', header=True, inferSchema=True)

In [86]:
# a. Create InvoiceYear as a derived column from Invoice date

df4=df3.select('InvoiceNo','StockCode','Description','Quantity','UnitPrice','CustomerID','Country',
    to_timestamp(f.col('InvoiceDate'), 'M/d/y H:m').alias('InvoiceDate')
)
df4=df4.withColumn("InvoiceYear", f.from_unixtime(f.unix_timestamp(df4.InvoiceDate), "yyyy"))
df4=df4.filter('InvoiceYear == 2011')
df4.show(3)
df4.printSchema()

+---------+---------+--------------------+--------+---------+----------+--------------+-------------------+-----------+
|InvoiceNo|StockCode|         Description|Quantity|UnitPrice|CustomerID|       Country|        InvoiceDate|InvoiceYear|
+---------+---------+--------------------+--------+---------+----------+--------------+-------------------+-----------+
|   539993|    22386|JUMBO BAG PINK PO...|      10|     1.95|     13313|United Kingdom|2011-01-04 10:00:00|       2011|
|   539993|    21499|  BLUE POLKADOT WRAP|      25|     0.42|     13313|United Kingdom|2011-01-04 10:00:00|       2011|
|   539993|    21498| RED RETROSPOT WRAP |      25|     0.42|     13313|United Kingdom|2011-01-04 10:00:00|       2011|
+---------+---------+--------------------+--------+---------+----------+--------------+-------------------+-----------+
only showing top 3 rows

root
 |-- InvoiceNo: string (nullable = true)
 |-- StockCode: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- 

In [87]:
# b. Filter all the customers with null value
df4=df4.filter("CustomerID is not NULL")
df4.show()

+---------+---------+--------------------+--------+---------+----------+--------------+-------------------+-----------+
|InvoiceNo|StockCode|         Description|Quantity|UnitPrice|CustomerID|       Country|        InvoiceDate|InvoiceYear|
+---------+---------+--------------------+--------+---------+----------+--------------+-------------------+-----------+
|   539993|    22386|JUMBO BAG PINK PO...|      10|     1.95|     13313|United Kingdom|2011-01-04 10:00:00|       2011|
|   539993|    21499|  BLUE POLKADOT WRAP|      25|     0.42|     13313|United Kingdom|2011-01-04 10:00:00|       2011|
|   539993|    21498| RED RETROSPOT WRAP |      25|     0.42|     13313|United Kingdom|2011-01-04 10:00:00|       2011|
|   539993|    22379|RECYCLING BAG RET...|       5|      2.1|     13313|United Kingdom|2011-01-04 10:00:00|       2011|
|   539993|    20718|RED RETROSPOT SHO...|      10|     1.25|     13313|United Kingdom|2011-01-04 10:00:00|       2011|
|   539993|   85099B|JUMBO BAG RED RET..

In [88]:
#c. SaleValue is derived as UnitPrice*Quantity
df4=df4.withColumn('SaleValue', round(abs(df4.Quantity * df4.UnitPrice),2))
df4.show(3)

+---------+---------+--------------------+--------+---------+----------+--------------+-------------------+-----------+---------+
|InvoiceNo|StockCode|         Description|Quantity|UnitPrice|CustomerID|       Country|        InvoiceDate|InvoiceYear|SaleValue|
+---------+---------+--------------------+--------+---------+----------+--------------+-------------------+-----------+---------+
|   539993|    22386|JUMBO BAG PINK PO...|      10|     1.95|     13313|United Kingdom|2011-01-04 10:00:00|       2011|     19.5|
|   539993|    21499|  BLUE POLKADOT WRAP|      25|     0.42|     13313|United Kingdom|2011-01-04 10:00:00|       2011|     10.5|
|   539993|    21498| RED RETROSPOT WRAP |      25|     0.42|     13313|United Kingdom|2011-01-04 10:00:00|       2011|     10.5|
+---------+---------+--------------------+--------+---------+----------+--------------+-------------------+-----------+---------+
only showing top 3 rows



In [113]:
#d.Arrange the data in DESC order of SaleValue
#e.Fetch the following data: CustomerId, TotalSaleValue, InvoiceYear,NumberofOrders

df5=df4.groupby('CustomerID').agg(round(sum("SaleValue"),2).alias("TotalSaleValue"), \
                                              count("CustomerID").alias("NumberofOrders")).sort(f.desc("TotalSaleValue"))


df5.limit(10).show()

+----------+--------------+--------------+
|CustomerID|TotalSaleValue|NumberofOrders|
+----------+--------------+--------------+
|     16446|      336942.1|             4|
|     14646|     272331.14|          2015|
|     18102|      235041.5|           415|
|     17450|     199590.57|           348|
|     12346|      154367.2|             2|
|     14911|     146358.75|          5585|
|     12415|     126103.61|           778|
|     14156|     121045.42|          1415|
|     16029|      97944.55|           242|
|     17511|      87211.98|           974|
+----------+--------------+--------------+



## 4. Create a streaming file format conversion pipeline using Filestreams to convert CSV files into PArquet files in real time.
#Streaming source: CSV(file source), Sink: Parquet(File sink)

In [8]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, IntegerType

if __name__ == "__main__":
    spark = SparkSession \
        .builder \
        .appName("File Streaming") \
        .getOrCreate()

    spark.sparkContext.setLogLevel("ERROR")

path = "C:/Users/91784/Spark/stream/source_csv_files/csv_files"

    mySchema = StructType([
        StructField("id", IntegerType(), True),
        StructField("name", StringType(), True),
        StructField("age", IntegerType(), True),

    df = spark.readStream.csv(path, header=True, schema=mySchema)

    query = df \
        .writeStream \
        .outputMode('append') \
        .trigger(processingTime='1 seconds')\
        .format('parquet') \
        .option("path", "C:/Users/91784/Spark/stream/Output/parquet_files") \
        .start()

query.awaitTermination()



## 5.

In [139]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, lit, concat

if __name__ == "__main__":
    
    spark = SparkSession\
        .builder\
        .appName("Rate Streaming")\
        .getOrCreate()
        
    spark.sparkContext.setLogLevel("ERROR")  
    spark.conf.set("spark.sql.shuffle.partitions", "1")
    spark.conf.set("spark.sql.streaming.checkpointLocation", "/C:/Users/Spark/stream/checkpoint/file_sink");
       
    df = spark.readStream.format("rate").option("rowsPerSecond", 5).load()
    
    
    resultDF = df.withColumn("value", concat(lit("Message: "), col("value"))) \
                 .selectExpr("CAST(timestamp AS STRING)", "CAST(value AS STRING)") \
                 .withColumnRenamed("timestamp", "ts")\
                 .withColumnRenamed("value", "message")   
    
    def process_data(df, epoch_id):
        
        # write to mysql
        df.write.format("jdbc") \
            .option("url", "jdbc:mysql://localhost/sparkdb?autoReConnect=true&useSSL=false") \
            .option("driver", "com.mysql.jdbc.Driver") \
            .option("dbtable", "rate_stream_1")  \
            .option("user", "root") \
            .option("password@123", "Mari") \
            .mode("append") \
            .save()            
    
        pass
    
    query = resultDF.writeStream.foreachBatch(process_data).start()
    query.awaitTermination()

    
